In [2]:
import re
import pandas as pd
import os
import math
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
# from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

# Воспользуемся собственным списком стоп-слов. stop_words = stopwords.words('russian') - как альтернатива.
with open ('stop_ru.txt', 'r', encoding='utf8') as stop_file:
    stop_words = [word.strip() for word in stop_file.readlines()]

# Подгружаем лемматизатор (будем брать начальную форму слова, благодаря pymorphy2).
morph = MorphAnalyzer()

# В предобработку включен выбор n-gram.
def data_preprocessing(data, k):

    # Очищаем данные.
    data = re.sub('[^A-Za-zА-Яа-я0-9]+', '  ', data)

    # Переводим слова в нижний регистр.
    data = data.lower()

    # Токенизируем корпус.
    tokens = word_tokenize(data, language='russian')

    # Удаляем стоп-слова для случая с юниграмами. Иначе оставляем.
    if k == 1:
        data = [word for word in tokens if word not in stop_words]
    else:
        data = tokens

    # Производим лемматизицаию.
    data = [morph.normal_forms(word)[0] for word in data]

    # Работаем с n-gram'ами.
    if k > 1:
        data = list(ngrams(data, k))

    # Возвращаем результат пользователю.
    return data


# Перед началом работы создадим папку для хранения результатов.
# os.mkdir("Result")

# Начинаем работу с файлами. Меняем директорию.
os.chdir("VK")
# Создаем список с именами обрабатываемых файлов.
LoF = os.listdir()
# Создаем data frame, в котором будем хранить результат.
df_w = pd.DataFrame()
# Начинаем работу в цикле по всем файлам.
for FileIndex in range(1):

    # Считываем данные файла.
    df = pd.read_excel(LoF[FileIndex])

    # Инициализурем юниграмы и биграмы.
    df_w['Text'] = df['Текст']
    df_w['Unigrams'] = df['Текст'].apply(lambda txt: data_preprocessing(txt, 1))
    df_w['Bigrams'] = df['Текст'].apply(lambda txt: data_preprocessing(txt, 2))

    # BoW для юниграм и биграм.
    df_w['Unigrams_BoW'] = df['Текст'].apply(lambda txt: FreqDist(data_preprocessing(txt, 1)))
    df_w['Bigrams_BoW'] = df['Текст'].apply(lambda txt: FreqDist(data_preprocessing(txt, 2)))

    # Получаем ключевые слова при помощи метода BoW.
    df_w['Unigrams_BoW_result'] = df['Текст'].apply(lambda txt: FreqDist(data_preprocessing(txt, 1)).most_common(5))
    df_w['Bigrams_BoW_result'] = df['Текст'].apply(lambda txt: FreqDist(data_preprocessing(txt, 2)).most_common(5))

    # TF-IDF
    # Создаем листы результатов.
    result_uni = []
    result_bi = []

    # Начинаем работу с каждым текстом по очереди.
    for txt in df_w['Text']:

        # Разбиваем текст на предложения.
        tmp = pd.DataFrame()
        tmp['Text'] = sent_tokenize(txt)
        tmp['Text'] = tmp['Text'].apply(lambda text: data_preprocessing(text, 1))
        tmp_list = tmp['Text'].to_list()
        for i in range(len(tmp_list)):
            tmp_list[i] = ' '.join(tmp_list[i])

        # TF-IDF для юниграмы.
        # Создаем векторайзер, после чего все преобразовывае к Data Frame'у.
        vectorizer_uni = TfidfVectorizer(analyzer='word', ngram_range=(1,1), stop_words = stop_words)
        vectorizer_uni.fit(tmp_list)
        tfidf_uni = vectorizer_uni.transform(tmp_list)
        vectors_uni = vectorizer_uni.get_feature_names()
        smatrix_uni = vectorizer_uni.transform(tmp_list)
        dense_uni = smatrix_uni.todense()
        dense_list_uni = dense_uni.tolist()
        df_tfidf_uni = pd.DataFrame(dense_list_uni, columns=vectors_uni)

        # Далее будем высчитывать топ-3 слова по TF-IDF.
        # После чего формируем строчку с результатом для каждого предложения.
        lst_words_uni = []
        tmp_uni = []
        for i in range(df_tfidf_uni.shape[0]):
            lst_words_uni.append(df_tfidf_uni.iloc[i].sort_values(ascending=False)[0:3])
            tmp_uni.append( 'KWUP ' + str(i) + ': ' + ', '.join(lst_words_uni[i].index))
        result_uni.append('. '.join(tmp_uni))

        # TF-IDF для биграмы.
        # Здесь все аналогично, только параметр ngram = (2,2).
        vectorizer_bi = TfidfVectorizer(analyzer='word', ngram_range=(2,2), stop_words = stop_words)
        vectorizer_bi.fit(tmp_list)
        tfidf_bi = vectorizer_bi.transform(tmp_list)
        vectors_bi = vectorizer_bi.get_feature_names()
        smatrix_bi = vectorizer_bi.transform(tmp_list)
        dense_bi = smatrix_bi.todense()
        dense_list_bi = dense_bi.tolist()
        df_tfidf_bi = pd.DataFrame(dense_list_bi, columns=vectors_bi)

        # Абсолютно аналогично с тем, что мы делали для юниграм.
        lst_words_bi = []
        tmp_bi = []
        for i in range(df_tfidf_bi.shape[0]):
            lst_words_bi.append(df_tfidf_bi.iloc[i].sort_values(ascending=False)[0:3])
            tmp_bi.append( 'KWBP ' + str(i) + ': ' + ', '.join(lst_words_bi[i].index))
        result_bi.append(' '.join(tmp_bi))

    # Заполняем колонку с результатами.
    df_w['TF-IDF.UNI'] = result_uni
    df_w['TF-IDF.BI'] = result_bi

    # Выгружаем файл.
    os.chdir("..")
    os.chdir("Result")
    df_w.to_excel(LoF[FileIndex])

# Выходим в исходную директорию.
os.chdir("..")
# from wordcloud import WordCloud - интересно попробовать!

c:\users\ender\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\ender\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\ender\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\ender\appdata\local\programs\python\python3